In [1]:
import numpy as np
from scipy.optimize import dual_annealing
import pulp
import pandas as pd
import os
import random
from wmape import WMAPE
from order_generator import OrderGenerator
from order_merger import OrderMerger
import math

In [2]:
random.seed(42)

In [4]:
# import allocations for lead day 5 and lead day 0 and calculate wmape values

allocation_dir = 'allocations'
merged_dir = 'merged'
lead_day = 5
soft_col = f'lead_{lead_day}'
hard_col = 'lead_0'

allocation_0 = pd.read_csv(f'{allocation_dir}/allocation_lead_day_0.csv')
allocation_5 = pd.read_csv(f'{allocation_dir}/allocation_lead_day_5.csv')

merged_5_0 = pd.read_csv(f'{merged_dir}/merged_lead_day_{lead_day}_0.csv')

_, wmape_site = WMAPE.calculate_wmape_site(merged_5_0, soft_col, hard_col)
_, wmape_global = WMAPE.calculate_wmape_global(merged_5_0, soft_col, hard_col)

print(f'wmape_global: {wmape_global}')
print(f'wmape_site: {wmape_site}')

wmape_global: 0.21075893420886477
wmape_site: 0.42063391842404346


In [5]:
def objective_function(current_allocation, orders_df, actual_orders_df):
    # Apply the current allocation to the orders dataframe
    orders_df['site'] = current_allocation

    # Aggregate the orders for lead day 5
    aggregated_5_df = OrderMerger.aggregate_items(orders_df, 5)
    aggregated_0_df = OrderMerger.aggregate_items(actual_orders_df, 0)

    # Merge the aggregated dataframes
    merged_df = OrderMerger.merge_allocation(aggregated_5_df, aggregated_0_df, 5, 0)

    # Calculate WMAPE
    _, wmape_site = WMAPE.calculate_wmape_site(merged_df, 'lead_5', 'lead_0')
    return wmape_site